## 全体の流れ
<ol>
<li>株価データを読み込む</li>
<li>データの全体像をつかむ</li>
<li>特徴量を追加する</li>
<li>学習データと検証データに分割する</li>
<li>データを整える</li>
<li>予測モデルの作成と交差検証</li>
<li>検証データによる予測と精度検証</li>
</ol>

## 株価データを読み込み目的変数を追加する

In [1]:
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
import matplotlib.pyplot as plt
import warnings
import yfinance as yf
import datetime
from dtw import dtw
from scipy.spatial.distance import euclidean
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import MinMaxScaler
warnings.simplefilter("ignore")
%matplotlib inline

/root/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
train_dates=["2012-01-01","2012-02-01","2012-03-01","2012-04-01","2012-05-01","2012-06-01","2012-07-01","2012-08-01","2012-09-01","2012-10-01","2012-11-01","2012-12-01",
             "2013-01-01","2013-02-01","2013-03-01","2013-04-01","2013-05-01","2013-06-01","2013-07-01","2013-08-01","2013-09-01","2013-10-01","2013-11-01","2013-12-01",
             "2014-01-01","2014-02-01","2014-03-01","2014-04-01","2014-05-01","2014-06-01","2014-07-01","2014-08-01","2014-09-01","2014-10-01","2014-11-01","2014-12-01",
             "2015-01-01","2015-02-01","2015-03-01","2015-04-01","2015-05-01","2015-06-01","2015-07-01","2015-08-01","2015-09-01","2015-10-01","2015-11-01","2015-12-01",
             "2016-01-01","2016-02-01","2016-03-01","2016-04-01","2016-05-01","2016-06-01","2016-07-01","2016-08-01","2016-09-01","2016-10-01","2016-11-01","2016-12-01",
             "2017-01-01","2017-02-01","2017-03-01","2017-04-01","2017-05-01","2017-06-01","2017-07-01","2017-08-01","2017-09-01","2017-10-01","2017-11-01","2017-12-01",
             "2018-01-01","2018-02-01","2018-03-01","2018-04-01","2018-05-01","2018-06-01","2018-07-01","2018-08-01","2018-09-01","2018-10-01","2018-11-01","2018-12-01",
             "2019-01-01","2019-02-01","2019-03-01","2019-04-01","2019-05-01","2019-06-01","2019-07-01","2019-08-01","2019-09-01","2019-10-01","2019-11-01","2019-12-01",
             "2020-01-01","2020-02-01","2020-03-01","2020-04-01","2020-05-01","2020-06-01","2020-07-01","2020-08-01","2020-09-01","2020-10-01","2020-11-01","2020-12-01"]
file_names=list(map(lambda s: s + '.csv', train_dates))

In [7]:
#日足データ
topix_close=[]
topix_index=[]
topix_volume=[]
MinMax_close=[]
MinMax_index=[]
for file in file_names[:48]:
        data=pd.read_csv(file,header=0)
        #正規化のクラスを準備
        ms = MinMaxScaler()
        #特徴量の最大値と最小値を計算し変換
        X = np.array(data["Close"]).reshape(-1,1)
        data["MinMax_Close"] = ms.fit_transform(X)#最小値0最大値1の正規化
        a=data["Close"][0]#日の始値
        b=data["MinMax_Close"][0]#日の始値
        data["MinMax_index"]=data["Close"]
        data["index"]=data["Close"]
        for i in range(len(data.index)):
            data["index"][i]=data["Close"][i]/a
            data["MinMax_index"][i]=data["MinMax_Close"][i]/b
        topix_close.append(data["Close"].tail(1).values)
        topix_index.append(data["index"].values)
        topix_volume.append(data["Volume"].values)
        MinMax_close.append(data["MinMax_Close"].values)
        MinMax_index.append(data["MinMax_index"].values)
df_topix_index=pd.DataFrame(topix_index,index=train_dates)
df_topix_close=pd.DataFrame(topix_close,index=train_dates)
df_topix_volume=pd.DataFrame(topix_volume,index=train_dates)
df_MinMax_close=pd.DataFrame(MinMax_close,index=train_dates)
df_MinMax_index=pd.DataFrame(MinMax_index,index=train_dates)
#可視化　
#plt.plot(df_topix_index.T)
#plt.plot(df_MinMax_index.T)

ValueError: Shape of passed values is (48, 23), indices imply (108, 23)

In [6]:
#15分足のデータ
file_15min=['15min/2012-01-04.csv', '15min/2012-01-05.csv', '15min/2012-01-06.csv', '15min/2012-01-10.csv', '15min/2012-01-11.csv', '15min/2012-01-12.csv', '15min/2012-01-13.csv', '15min/2012-01-16.csv', '15min/2012-01-17.csv', '15min/2012-01-18.csv', '15min/2012-01-19.csv', '15min/2012-01-20.csv', '15min/2012-01-23.csv', '15min/2012-01-24.csv', '15min/2012-01-25.csv', '15min/2012-01-26.csv', '15min/2012-01-27.csv', '15min/2012-01-30.csv', '15min/2012-01-31.csv', '15min/2012-02-01.csv', '15min/2012-02-02.csv', '15min/2012-02-03.csv', '15min/2012-02-06.csv', '15min/2012-02-07.csv', '15min/2012-02-08.csv', '15min/2012-02-09.csv', '15min/2012-02-10.csv', '15min/2012-02-13.csv', '15min/2012-02-14.csv', '15min/2012-02-15.csv', '15min/2012-02-16.csv', '15min/2012-02-17.csv', '15min/2012-02-20.csv', '15min/2012-02-21.csv', '15min/2012-02-22.csv', '15min/2012-02-23.csv', '15min/2012-02-24.csv', '15min/2012-02-27.csv', '15min/2012-02-28.csv', '15min/2012-02-29.csv', '15min/2012-03-01.csv', '15min/2012-03-02.csv', '15min/2012-03-05.csv', '15min/2012-03-06.csv', '15min/2012-03-07.csv', '15min/2012-03-08.csv', '15min/2012-03-09.csv', '15min/2012-03-12.csv', '15min/2012-03-13.csv', '15min/2012-03-14.csv', '15min/2012-03-15.csv', '15min/2012-03-16.csv', '15min/2012-03-19.csv', '15min/2012-03-21.csv', '15min/2012-03-22.csv', '15min/2012-03-23.csv', '15min/2012-03-26.csv', '15min/2012-03-27.csv', '15min/2012-03-28.csv', '15min/2012-03-29.csv', '15min/2012-03-30.csv', '15min/2012-04-01.csv', '15min/2012-04-02.csv', '15min/2012-04-03.csv', '15min/2012-04-04.csv', '15min/2012-04-05.csv', '15min/2012-04-06.csv', '15min/2012-04-09.csv', '15min/2012-04-10.csv', '15min/2012-04-11.csv', '15min/2012-04-12.csv', '15min/2012-04-13.csv', '15min/2012-04-16.csv', '15min/2012-04-17.csv', '15min/2012-04-18.csv', '15min/2012-04-19.csv', '15min/2012-04-20.csv', '15min/2012-04-23.csv', '15min/2012-04-24.csv', '15min/2012-04-25.csv', '15min/2012-04-26.csv', '15min/2012-04-27.csv', '15min/2012-05-01.csv', '15min/2012-05-02.csv', '15min/2012-05-07.csv', '15min/2012-05-08.csv', '15min/2012-05-09.csv', '15min/2012-05-10.csv', '15min/2012-05-11.csv', '15min/2012-05-14.csv', '15min/2012-05-15.csv', '15min/2012-05-16.csv', '15min/2012-05-17.csv', '15min/2012-05-18.csv', '15min/2012-05-21.csv', '15min/2012-05-22.csv', '15min/2012-05-23.csv', '15min/2012-05-24.csv', '15min/2012-05-25.csv', '15min/2012-05-28.csv', '15min/2012-05-29.csv', '15min/2012-05-30.csv', '15min/2012-05-31.csv', '15min/2012-06-01.csv', '15min/2012-06-04.csv', '15min/2012-06-05.csv', '15min/2012-06-06.csv', '15min/2012-06-07.csv', '15min/2012-06-08.csv', '15min/2012-06-11.csv', '15min/2012-06-12.csv', '15min/2012-06-13.csv', '15min/2012-06-14.csv', '15min/2012-06-15.csv', '15min/2012-06-18.csv', '15min/2012-06-19.csv', '15min/2012-06-20.csv', '15min/2012-06-21.csv', '15min/2012-06-22.csv', '15min/2012-06-25.csv', '15min/2012-06-26.csv', '15min/2012-06-27.csv', '15min/2012-06-28.csv', '15min/2012-06-29.csv', '15min/2012-07-01.csv', '15min/2012-07-02.csv', '15min/2012-07-03.csv', '15min/2012-07-04.csv', '15min/2012-07-05.csv', '15min/2012-07-06.csv', '15min/2012-07-09.csv', '15min/2012-07-10.csv', '15min/2012-07-11.csv', '15min/2012-07-12.csv', '15min/2012-07-13.csv', '15min/2012-07-17.csv', '15min/2012-07-18.csv', '15min/2012-07-19.csv', '15min/2012-07-20.csv', '15min/2012-07-23.csv', '15min/2012-07-24.csv', '15min/2012-07-25.csv', '15min/2012-07-26.csv', '15min/2012-07-27.csv', '15min/2012-07-30.csv', '15min/2012-07-31.csv', '15min/2012-08-01.csv', '15min/2012-08-02.csv', '15min/2012-08-03.csv', '15min/2012-08-06.csv', '15min/2012-08-07.csv', '15min/2012-08-08.csv', '15min/2012-08-09.csv', '15min/2012-08-10.csv', '15min/2012-08-13.csv', '15min/2012-08-14.csv', '15min/2012-08-15.csv', '15min/2012-08-16.csv', '15min/2012-08-17.csv', '15min/2012-08-20.csv', '15min/2012-08-21.csv', '15min/2012-08-22.csv', '15min/2012-08-23.csv', '15min/2012-08-24.csv', '15min/2012-08-27.csv', '15min/2012-08-28.csv', '15min/2012-08-29.csv', '15min/2012-08-30.csv', '15min/2012-08-31.csv', '15min/2012-09-01.csv', '15min/2012-09-03.csv', '15min/2012-09-04.csv', '15min/2012-09-05.csv', '15min/2012-09-06.csv', '15min/2012-09-07.csv', '15min/2012-09-10.csv', '15min/2012-09-11.csv', '15min/2012-09-12.csv', '15min/2012-09-13.csv', '15min/2012-09-14.csv', '15min/2012-09-18.csv', '15min/2012-09-19.csv', '15min/2012-09-20.csv', '15min/2012-09-21.csv', '15min/2012-09-24.csv', '15min/2012-09-25.csv', '15min/2012-09-26.csv', '15min/2012-09-27.csv', '15min/2012-09-28.csv', '15min/2012-10-01.csv', '15min/2012-10-02.csv', '15min/2012-10-03.csv', '15min/2012-10-04.csv', '15min/2012-10-05.csv', '15min/2012-10-09.csv', '15min/2012-10-10.csv', '15min/2012-10-11.csv', '15min/2012-10-12.csv', '15min/2012-10-15.csv', '15min/2012-10-16.csv', '15min/2012-10-17.csv', '15min/2012-10-18.csv', '15min/2012-10-19.csv', '15min/2012-10-22.csv', '15min/2012-10-23.csv', '15min/2012-10-24.csv', '15min/2012-10-25.csv', '15min/2012-10-26.csv', '15min/2012-10-29.csv', '15min/2012-10-30.csv', '15min/2012-10-31.csv', '15min/2012-11-01.csv', '15min/2012-11-02.csv', '15min/2012-11-05.csv', '15min/2012-11-06.csv', '15min/2012-11-07.csv', '15min/2012-11-08.csv', '15min/2012-11-09.csv', '15min/2012-11-12.csv', '15min/2012-11-13.csv', '15min/2012-11-14.csv', '15min/2012-11-15.csv', '15min/2012-11-16.csv', '15min/2012-11-19.csv', '15min/2012-11-20.csv', '15min/2012-11-21.csv', '15min/2012-11-22.csv', '15min/2012-11-26.csv', '15min/2012-11-27.csv', '15min/2012-11-28.csv', '15min/2012-11-29.csv', '15min/2012-11-30.csv', '15min/2012-12-01.csv', '15min/2012-12-03.csv', '15min/2012-12-04.csv', '15min/2012-12-05.csv', '15min/2012-12-06.csv', '15min/2012-12-07.csv', '15min/2012-12-10.csv', '15min/2012-12-11.csv', '15min/2012-12-12.csv', '15min/2012-12-13.csv', '15min/2012-12-14.csv', '15min/2012-12-17.csv', '15min/2012-12-18.csv', '15min/2012-12-19.csv', '15min/2012-12-20.csv', '15min/2012-12-21.csv', '15min/2012-12-25.csv', '15min/2012-12-26.csv', '15min/2012-12-27.csv', '15min/2012-12-28.csv', '15min/2013-01-04.csv', '15min/2013-01-07.csv', '15min/2013-01-08.csv', '15min/2013-01-09.csv', '15min/2013-01-10.csv', '15min/2013-01-11.csv', '15min/2013-01-15.csv', '15min/2013-01-16.csv', '15min/2013-01-17.csv', '15min/2013-01-18.csv', '15min/2013-01-21.csv', '15min/2013-01-22.csv', '15min/2013-01-23.csv', '15min/2013-01-24.csv', '15min/2013-01-25.csv', '15min/2013-01-28.csv', '15min/2013-01-29.csv', '15min/2013-01-30.csv', '15min/2013-01-31.csv', '15min/2013-02-01.csv', '15min/2013-02-04.csv', '15min/2013-02-05.csv', '15min/2013-02-06.csv', '15min/2013-02-07.csv', '15min/2013-02-08.csv', '15min/2013-02-12.csv', '15min/2013-02-13.csv', '15min/2013-02-14.csv', '15min/2013-02-15.csv', '15min/2013-02-18.csv', '15min/2013-02-19.csv', '15min/2013-02-20.csv', '15min/2013-02-21.csv', '15min/2013-02-22.csv', '15min/2013-02-25.csv', '15min/2013-02-26.csv', '15min/2013-02-27.csv', '15min/2013-02-28.csv', '15min/2013-03-01.csv', '15min/2013-03-04.csv', '15min/2013-03-05.csv', '15min/2013-03-06.csv', '15min/2013-03-07.csv', '15min/2013-03-08.csv', '15min/2013-03-11.csv', '15min/2013-03-12.csv', '15min/2013-03-13.csv', '15min/2013-03-14.csv', '15min/2013-03-15.csv', '15min/2013-03-18.csv', '15min/2013-03-19.csv', '15min/2013-03-21.csv', '15min/2013-03-22.csv', '15min/2013-03-25.csv', '15min/2013-03-26.csv', '15min/2013-03-27.csv', '15min/2013-03-28.csv', '15min/2013-03-29.csv', '15min/2013-04-01.csv', '15min/2013-04-02.csv', '15min/2013-04-03.csv', '15min/2013-04-04.csv', '15min/2013-04-05.csv', '15min/2013-04-08.csv', '15min/2013-04-09.csv', '15min/2013-04-10.csv', '15min/2013-04-11.csv', '15min/2013-04-12.csv', '15min/2013-04-15.csv', '15min/2013-04-16.csv', '15min/2013-04-17.csv', '15min/2013-04-18.csv', '15min/2013-04-19.csv', '15min/2013-04-22.csv', '15min/2013-04-23.csv', '15min/2013-04-24.csv', '15min/2013-04-25.csv', '15min/2013-04-26.csv', '15min/2013-04-30.csv', '15min/2013-05-01.csv', '15min/2013-05-02.csv', '15min/2013-05-07.csv', '15min/2013-05-08.csv', '15min/2013-05-09.csv', '15min/2013-05-10.csv', '15min/2013-05-13.csv', '15min/2013-05-14.csv', '15min/2013-05-15.csv', '15min/2013-05-16.csv', '15min/2013-05-17.csv', '15min/2013-05-20.csv', '15min/2013-05-21.csv', '15min/2013-05-22.csv', '15min/2013-05-23.csv', '15min/2013-05-24.csv', '15min/2013-05-27.csv', '15min/2013-05-28.csv', '15min/2013-05-29.csv', '15min/2013-05-30.csv', '15min/2013-05-31.csv', '15min/2013-06-03.csv', '15min/2013-06-04.csv', '15min/2013-06-05.csv', '15min/2013-06-06.csv', '15min/2013-06-07.csv', '15min/2013-06-10.csv', '15min/2013-06-11.csv', '15min/2013-06-12.csv', '15min/2013-06-13.csv', '15min/2013-06-14.csv', '15min/2013-06-17.csv', '15min/2013-06-18.csv', '15min/2013-06-19.csv', '15min/2013-06-20.csv', '15min/2013-06-21.csv', '15min/2013-06-24.csv', '15min/2013-06-25.csv', '15min/2013-06-26.csv', '15min/2013-06-27.csv', '15min/2013-06-28.csv', '15min/2013-07-01.csv', '15min/2013-07-02.csv', '15min/2013-07-03.csv', '15min/2013-07-04.csv', '15min/2013-07-05.csv', '15min/2013-07-08.csv', '15min/2013-07-09.csv', '15min/2013-07-10.csv', '15min/2013-07-11.csv', '15min/2013-07-12.csv', '15min/2013-07-16.csv', '15min/2013-07-17.csv', '15min/2013-07-18.csv', '15min/2013-07-19.csv', '15min/2013-07-22.csv', '15min/2013-07-23.csv', '15min/2013-07-24.csv', '15min/2013-07-25.csv', '15min/2013-07-26.csv', '15min/2013-07-29.csv', '15min/2013-07-30.csv', '15min/2013-07-31.csv', '15min/2013-08-01.csv', '15min/2013-08-02.csv', '15min/2013-08-05.csv', '15min/2013-08-06.csv', '15min/2013-08-07.csv', '15min/2013-08-08.csv', '15min/2013-08-09.csv', '15min/2013-08-12.csv', '15min/2013-08-13.csv', '15min/2013-08-14.csv', '15min/2013-08-15.csv', '15min/2013-08-16.csv', '15min/2013-08-19.csv', '15min/2013-08-20.csv', '15min/2013-08-21.csv', '15min/2013-08-22.csv', '15min/2013-08-23.csv', '15min/2013-08-26.csv', '15min/2013-08-27.csv', '15min/2013-08-28.csv', '15min/2013-08-29.csv', '15min/2013-08-30.csv', '15min/2013-09-02.csv', 
            '15min/2013-09-03.csv', '15min/2013-09-04.csv', '15min/2013-09-05.csv', '15min/2013-09-06.csv', '15min/2013-09-09.csv', '15min/2013-09-10.csv', '15min/2013-09-11.csv', '15min/2013-09-12.csv', '15min/2013-09-13.csv', '15min/2013-09-17.csv', '15min/2013-09-18.csv', '15min/2013-09-19.csv', '15min/2013-09-20.csv', '15min/2013-09-24.csv', '15min/2013-09-25.csv', '15min/2013-09-26.csv', '15min/2013-09-27.csv', '15min/2013-09-30.csv', '15min/2013-10-01.csv', '15min/2013-10-02.csv', '15min/2013-10-03.csv', '15min/2013-10-04.csv', '15min/2013-10-07.csv', '15min/2013-10-08.csv', '15min/2013-10-09.csv', '15min/2013-10-10.csv', '15min/2013-10-11.csv', '15min/2013-10-15.csv', '15min/2013-10-16.csv', '15min/2013-10-17.csv', '15min/2013-10-18.csv', '15min/2013-10-21.csv', '15min/2013-10-22.csv', '15min/2013-10-23.csv', '15min/2013-10-24.csv', '15min/2013-10-25.csv', '15min/2013-10-28.csv', '15min/2013-10-29.csv', '15min/2013-10-30.csv', '15min/2013-10-31.csv', '15min/2013-11-01.csv', '15min/2013-11-05.csv', '15min/2013-11-06.csv', '15min/2013-11-07.csv', '15min/2013-11-08.csv', '15min/2013-11-11.csv', '15min/2013-11-12.csv', '15min/2013-11-13.csv', '15min/2013-11-14.csv', '15min/2013-11-15.csv', '15min/2013-11-18.csv', '15min/2013-11-19.csv', '15min/2013-11-20.csv', '15min/2013-11-21.csv', '15min/2013-11-22.csv', '15min/2013-11-25.csv', '15min/2013-11-26.csv', '15min/2013-11-27.csv', '15min/2013-11-28.csv', '15min/2013-11-29.csv', '15min/2013-12-02.csv', '15min/2013-12-03.csv', '15min/2013-12-04.csv', '15min/2013-12-05.csv', '15min/2013-12-06.csv', '15min/2013-12-09.csv', '15min/2013-12-10.csv', '15min/2013-12-11.csv', '15min/2013-12-12.csv', '15min/2013-12-13.csv', '15min/2013-12-16.csv', '15min/2013-12-17.csv', '15min/2013-12-18.csv', '15min/2013-12-19.csv', '15min/2013-12-20.csv', '15min/2013-12-24.csv', '15min/2013-12-25.csv', '15min/2013-12-26.csv', '15min/2013-12-27.csv', '15min/2013-12-30.csv', '15min/2014-01-06.csv', '15min/2014-01-07.csv', '15min/2014-01-08.csv', '15min/2014-01-09.csv', '15min/2014-01-10.csv', '15min/2014-01-14.csv', '15min/2014-01-15.csv', '15min/2014-01-16.csv', '15min/2014-01-17.csv', '15min/2014-01-20.csv', '15min/2014-01-21.csv', '15min/2014-01-22.csv', '15min/2014-01-23.csv', '15min/2014-01-24.csv', '15min/2014-01-27.csv', '15min/2014-01-28.csv', '15min/2014-01-29.csv', '15min/2014-01-30.csv', '15min/2014-01-31.csv', '15min/2014-02-03.csv', '15min/2014-02-04.csv', '15min/2014-02-05.csv', '15min/2014-02-06.csv', '15min/2014-02-07.csv', '15min/2014-02-10.csv', '15min/2014-02-12.csv', '15min/2014-02-13.csv', '15min/2014-02-14.csv', '15min/2014-02-17.csv', '15min/2014-02-18.csv', '15min/2014-02-19.csv', '15min/2014-02-20.csv', '15min/2014-02-21.csv', '15min/2014-02-24.csv', '15min/2014-02-25.csv', '15min/2014-02-26.csv', '15min/2014-02-27.csv', '15min/2014-02-28.csv', '15min/2014-03-03.csv', '15min/2014-03-04.csv', '15min/2014-03-05.csv', '15min/2014-03-06.csv', '15min/2014-03-07.csv', '15min/2014-03-10.csv', '15min/2014-03-11.csv', '15min/2014-03-12.csv', '15min/2014-03-13.csv', '15min/2014-03-14.csv', '15min/2014-03-17.csv', '15min/2014-03-18.csv', '15min/2014-03-19.csv', '15min/2014-03-20.csv', '15min/2014-03-24.csv', '15min/2014-03-25.csv', '15min/2014-03-26.csv', '15min/2014-03-27.csv', '15min/2014-03-28.csv', '15min/2014-03-31.csv', '15min/2014-04-01.csv', '15min/2014-04-02.csv', '15min/2014-04-03.csv', '15min/2014-04-04.csv', '15min/2014-04-07.csv', '15min/2014-04-08.csv', '15min/2014-04-09.csv', '15min/2014-04-10.csv', '15min/2014-04-11.csv', '15min/2014-04-14.csv', '15min/2014-04-15.csv', '15min/2014-04-16.csv', '15min/2014-04-17.csv', '15min/2014-04-18.csv', '15min/2014-04-21.csv', '15min/2014-04-22.csv', '15min/2014-04-23.csv', '15min/2014-04-24.csv', '15min/2014-04-25.csv', '15min/2014-04-28.csv', '15min/2014-04-30.csv', '15min/2014-05-01.csv', '15min/2014-05-02.csv', '15min/2014-05-07.csv', '15min/2014-05-08.csv', '15min/2014-05-09.csv', '15min/2014-05-12.csv', '15min/2014-05-13.csv', '15min/2014-05-14.csv', '15min/2014-05-15.csv', '15min/2014-05-16.csv', '15min/2014-05-19.csv', '15min/2014-05-20.csv', '15min/2014-05-21.csv', '15min/2014-05-22.csv', '15min/2014-05-23.csv', '15min/2014-05-26.csv', '15min/2014-05-27.csv', '15min/2014-05-28.csv', '15min/2014-05-29.csv', '15min/2014-05-30.csv', '15min/2014-06-02.csv', '15min/2014-06-03.csv', '15min/2014-06-04.csv', '15min/2014-06-05.csv', '15min/2014-06-06.csv', '15min/2014-06-09.csv', '15min/2014-06-10.csv', '15min/2014-06-11.csv', '15min/2014-06-12.csv', '15min/2014-06-13.csv', '15min/2014-06-16.csv', '15min/2014-06-17.csv', '15min/2014-06-18.csv', '15min/2014-06-19.csv', '15min/2014-06-20.csv', '15min/2014-06-23.csv', '15min/2014-06-24.csv', '15min/2014-06-25.csv', '15min/2014-06-26.csv', '15min/2014-06-27.csv', '15min/2014-06-30.csv', '15min/2014-07-01.csv', '15min/2014-07-02.csv', '15min/2014-07-03.csv', '15min/2014-07-04.csv', '15min/2014-07-07.csv', '15min/2014-07-08.csv', '15min/2014-07-09.csv', '15min/2014-07-10.csv', '15min/2014-07-11.csv', '15min/2014-07-14.csv', '15min/2014-07-15.csv', '15min/2014-07-16.csv', '15min/2014-07-17.csv', '15min/2014-07-18.csv', '15min/2014-07-22.csv', '15min/2014-07-23.csv', '15min/2014-07-24.csv', '15min/2014-07-25.csv', '15min/2014-07-28.csv', '15min/2014-07-29.csv', '15min/2014-07-30.csv', '15min/2014-07-31.csv', '15min/2014-08-01.csv', '15min/2014-08-04.csv', '15min/2014-08-05.csv', '15min/2014-08-06.csv', '15min/2014-08-07.csv', '15min/2014-08-08.csv', '15min/2014-08-11.csv', '15min/2014-08-12.csv', '15min/2014-08-13.csv', '15min/2014-08-14.csv', '15min/2014-08-15.csv', '15min/2014-08-18.csv', '15min/2014-08-19.csv', '15min/2014-08-20.csv', '15min/2014-08-21.csv', '15min/2014-08-22.csv', '15min/2014-08-25.csv', '15min/2014-08-26.csv', '15min/2014-08-27.csv', '15min/2014-08-28.csv', '15min/2014-08-29.csv', '15min/2014-09-01.csv', '15min/2014-09-02.csv', '15min/2014-09-03.csv', '15min/2014-09-04.csv', '15min/2014-09-05.csv', '15min/2014-09-08.csv', '15min/2014-09-09.csv', '15min/2014-09-10.csv', '15min/2014-09-11.csv', '15min/2014-09-12.csv', '15min/2014-09-16.csv', '15min/2014-09-17.csv', '15min/2014-09-18.csv', '15min/2014-09-19.csv', '15min/2014-09-22.csv', '15min/2014-09-24.csv', '15min/2014-09-25.csv', '15min/2014-09-26.csv', '15min/2014-09-29.csv', '15min/2014-09-30.csv', '15min/2014-10-01.csv', '15min/2014-10-02.csv', '15min/2014-10-03.csv', '15min/2014-10-06.csv', '15min/2014-10-07.csv', '15min/2014-10-08.csv', '15min/2014-10-09.csv', '15min/2014-10-10.csv', '15min/2014-10-14.csv', '15min/2014-10-15.csv', '15min/2014-10-16.csv', '15min/2014-10-17.csv', '15min/2014-10-20.csv', '15min/2014-10-21.csv', '15min/2014-10-22.csv', '15min/2014-10-23.csv', '15min/2014-10-24.csv', '15min/2014-10-27.csv', '15min/2014-10-28.csv', '15min/2014-10-29.csv', '15min/2014-10-30.csv', '15min/2014-10-31.csv', '15min/2014-11-01.csv', '15min/2014-11-04.csv', '15min/2014-11-05.csv', '15min/2014-11-06.csv', '15min/2014-11-07.csv', '15min/2014-11-10.csv', '15min/2014-11-11.csv', '15min/2014-11-12.csv', '15min/2014-11-13.csv', '15min/2014-11-14.csv', '15min/2014-11-17.csv', '15min/2014-11-18.csv', '15min/2014-11-19.csv', '15min/2014-11-20.csv', '15min/2014-11-21.csv', '15min/2014-11-25.csv', '15min/2014-11-26.csv', '15min/2014-11-27.csv', '15min/2014-11-28.csv', '15min/2014-12-01.csv', '15min/2014-12-02.csv', '15min/2014-12-03.csv', '15min/2014-12-04.csv', '15min/2014-12-05.csv', '15min/2014-12-08.csv', '15min/2014-12-09.csv', '15min/2014-12-10.csv', '15min/2014-12-11.csv', '15min/2014-12-12.csv', '15min/2014-12-15.csv', '15min/2014-12-16.csv', '15min/2014-12-17.csv', '15min/2014-12-18.csv', '15min/2014-12-19.csv', '15min/2014-12-22.csv', '15min/2014-12-24.csv', '15min/2014-12-25.csv', '15min/2014-12-26.csv', '15min/2014-12-29.csv', '15min/2014-12-30.csv', '15min/2015-01-05.csv', '15min/2015-01-06.csv', '15min/2015-01-07.csv', '15min/2015-01-08.csv', '15min/2015-01-09.csv', '15min/2015-01-13.csv', '15min/2015-01-14.csv', '15min/2015-01-15.csv', '15min/2015-01-16.csv', '15min/2015-01-19.csv', '15min/2015-01-20.csv', '15min/2015-01-21.csv', '15min/2015-01-22.csv', '15min/2015-01-23.csv', '15min/2015-01-26.csv', '15min/2015-01-27.csv', '15min/2015-01-28.csv', '15min/2015-01-29.csv', '15min/2015-01-30.csv', '15min/2015-02-01.csv', '15min/2015-02-02.csv', '15min/2015-02-03.csv', '15min/2015-02-04.csv', '15min/2015-02-05.csv', '15min/2015-02-06.csv', '15min/2015-02-09.csv', '15min/2015-02-10.csv', '15min/2015-02-12.csv', '15min/2015-02-13.csv', '15min/2015-02-16.csv', '15min/2015-02-17.csv', '15min/2015-02-18.csv', '15min/2015-02-19.csv', '15min/2015-02-20.csv', '15min/2015-02-23.csv', '15min/2015-02-24.csv', '15min/2015-02-25.csv', '15min/2015-02-26.csv', '15min/2015-02-27.csv', '15min/2015-03-01.csv', '15min/2015-03-02.csv', '15min/2015-03-03.csv', '15min/2015-03-04.csv', '15min/2015-03-05.csv', '15min/2015-03-06.csv', '15min/2015-03-09.csv', '15min/2015-03-10.csv', '15min/2015-03-11.csv', '15min/2015-03-12.csv', '15min/2015-03-13.csv', '15min/2015-03-16.csv', '15min/2015-03-17.csv', '15min/2015-03-18.csv', '15min/2015-03-19.csv', '15min/2015-03-20.csv', '15min/2015-03-23.csv', '15min/2015-03-24.csv', '15min/2015-03-25.csv', '15min/2015-03-26.csv', '15min/2015-03-27.csv', '15min/2015-03-30.csv', '15min/2015-03-31.csv', '15min/2015-04-01.csv', '15min/2015-04-02.csv', '15min/2015-04-03.csv', '15min/2015-04-06.csv', '15min/2015-04-07.csv', '15min/2015-04-08.csv', '15min/2015-04-09.csv', '15min/2015-04-10.csv', '15min/2015-04-13.csv', '15min/2015-04-14.csv', '15min/2015-04-15.csv', '15min/2015-04-16.csv', '15min/2015-04-17.csv', '15min/2015-04-20.csv', '15min/2015-04-21.csv', '15min/2015-04-22.csv', '15min/2015-04-23.csv', '15min/2015-04-24.csv', '15min/2015-04-27.csv', '15min/2015-04-28.csv', '15min/2015-04-30.csv', '15min/2015-05-01.csv', '15min/2015-05-07.csv', '15min/2015-05-08.csv', '15min/2015-05-11.csv', '15min/2015-05-12.csv', '15min/2015-05-13.csv', '15min/2015-05-14.csv', '15min/2015-05-15.csv', '15min/2015-05-18.csv', 
            '15min/2015-05-19.csv', '15min/2015-05-20.csv', '15min/2015-05-21.csv', '15min/2015-05-22.csv', '15min/2015-05-25.csv', '15min/2015-05-26.csv', '15min/2015-05-27.csv', '15min/2015-05-28.csv', '15min/2015-05-29.csv', '15min/2015-06-01.csv', '15min/2015-06-02.csv', '15min/2015-06-03.csv', '15min/2015-06-04.csv', '15min/2015-06-05.csv', '15min/2015-06-08.csv', '15min/2015-06-09.csv', '15min/2015-06-10.csv', '15min/2015-06-11.csv', '15min/2015-06-12.csv', '15min/2015-06-15.csv', '15min/2015-06-16.csv', '15min/2015-06-17.csv', '15min/2015-06-18.csv', '15min/2015-06-19.csv', '15min/2015-06-22.csv', '15min/2015-06-23.csv', '15min/2015-06-24.csv', '15min/2015-06-25.csv', '15min/2015-06-26.csv', '15min/2015-06-29.csv', '15min/2015-06-30.csv', '15min/2015-07-01.csv', '15min/2015-07-02.csv', '15min/2015-07-03.csv', '15min/2015-07-06.csv', '15min/2015-07-07.csv', '15min/2015-07-08.csv', '15min/2015-07-09.csv', '15min/2015-07-10.csv', '15min/2015-07-13.csv', '15min/2015-07-14.csv', '15min/2015-07-15.csv', '15min/2015-07-16.csv', '15min/2015-07-17.csv', '15min/2015-07-21.csv', '15min/2015-07-22.csv', '15min/2015-07-23.csv', '15min/2015-07-24.csv', '15min/2015-07-27.csv', '15min/2015-07-28.csv', '15min/2015-07-29.csv', '15min/2015-07-30.csv', '15min/2015-07-31.csv', '15min/2015-08-01.csv', '15min/2015-08-03.csv', '15min/2015-08-04.csv', '15min/2015-08-05.csv', '15min/2015-08-06.csv', '15min/2015-08-07.csv', '15min/2015-08-10.csv', '15min/2015-08-11.csv', '15min/2015-08-12.csv', '15min/2015-08-13.csv', '15min/2015-08-14.csv', '15min/2015-08-17.csv', '15min/2015-08-18.csv', '15min/2015-08-19.csv', '15min/2015-08-20.csv', '15min/2015-08-21.csv', '15min/2015-08-24.csv', '15min/2015-08-25.csv', '15min/2015-08-26.csv', '15min/2015-08-27.csv', '15min/2015-08-28.csv', '15min/2015-08-31.csv', '15min/2015-09-01.csv', '15min/2015-09-02.csv', '15min/2015-09-03.csv', '15min/2015-09-04.csv', '15min/2015-09-07.csv', '15min/2015-09-08.csv', '15min/2015-09-09.csv', '15min/2015-09-10.csv', '15min/2015-09-11.csv', '15min/2015-09-14.csv', '15min/2015-09-15.csv', '15min/2015-09-16.csv', '15min/2015-09-17.csv', '15min/2015-09-18.csv', '15min/2015-09-24.csv', '15min/2015-09-25.csv', '15min/2015-09-28.csv', '15min/2015-09-29.csv', '15min/2015-09-30.csv', '15min/2015-10-01.csv', '15min/2015-10-02.csv', '15min/2015-10-05.csv', '15min/2015-10-06.csv', '15min/2015-10-07.csv', '15min/2015-10-08.csv', '15min/2015-10-09.csv', '15min/2015-10-13.csv', '15min/2015-10-14.csv', '15min/2015-10-15.csv', '15min/2015-10-16.csv', '15min/2015-10-19.csv', '15min/2015-10-20.csv', '15min/2015-10-21.csv', '15min/2015-10-22.csv', '15min/2015-10-23.csv', '15min/2015-10-26.csv', '15min/2015-10-27.csv', '15min/2015-10-28.csv', '15min/2015-10-29.csv', '15min/2015-10-30.csv', '15min/2015-11-01.csv', '15min/2015-11-02.csv', '15min/2015-11-04.csv', '15min/2015-11-05.csv', '15min/2015-11-06.csv', '15min/2015-11-09.csv', '15min/2015-11-10.csv', '15min/2015-11-11.csv', '15min/2015-11-12.csv', '15min/2015-11-13.csv', '15min/2015-11-16.csv', '15min/2015-11-17.csv', '15min/2015-11-18.csv', '15min/2015-11-19.csv', '15min/2015-11-20.csv', '15min/2015-11-24.csv', '15min/2015-11-25.csv', '15min/2015-11-26.csv', '15min/2015-11-27.csv', '15min/2015-11-30.csv', '15min/2015-12-01.csv', '15min/2015-12-02.csv', '15min/2015-12-03.csv', '15min/2015-12-04.csv', '15min/2015-12-07.csv', '15min/2015-12-08.csv', '15min/2015-12-09.csv', '15min/2015-12-10.csv', '15min/2015-12-11.csv', '15min/2015-12-14.csv', '15min/2015-12-15.csv', '15min/2015-12-16.csv', '15min/2015-12-17.csv', '15min/2015-12-18.csv', '15min/2015-12-21.csv', '15min/2015-12-22.csv', '15min/2015-12-24.csv', '15min/2015-12-25.csv', '15min/2015-12-28.csv', '15min/2015-12-29.csv', '15min/2015-12-30.csv']
train15min_data_date=[]
import os
for i in range(len(file_15min)):
    basename_without_ext = os.path.splitext(os.path.basename(file_15min[i]))[0]
    train15min_data_date.append(basename_without_ext)
topix15min_index=[]
topix15min_close=[]
topix15min_volume=[]
MinMax15min_index=[]
MinMax15min_close=[]
for file in file_15min:
        data=pd.read_csv(file,header=0)
        #正規化のクラスを準備
        ms = MinMaxScaler()
        #特徴量の最大値と最小値を計算し変換
        X = np.array(data["Close"]).reshape(-1,1)
        data["MinMax_Close"] = ms.fit_transform(X)#最小値0最大値1の正規化
        a=data["Close"][0]#日の始値
        b=data["MinMax_Close"][0]#日の始値
        data["MinMax_index"]=data["Close"]
        data["index"]=data["Close"]
        for i in range(len(data.index)):
            data["index"][i]=data["Close"][i]/a
            data["MinMax_index"][i]=data["MinMax_Close"][i]/b
        #data.to_csv(file, encoding="shift_jis")
        topix15min_close.append(data["Close"].tail(1).values)
        topix15min_index.append(data["index"].values)
        topix15min_volume.append(data["Volume"].values)
        MinMax15min_close.append(data["MinMax_Close"].values)
        MinMax15min_index.append(data["MinMax_index"].values)
df_topix15min_index=pd.DataFrame(topix15min_index,index=train15min_data_date)
df_topix15min_close=pd.DataFrame(topix15min_close,index=train15min_data_date)
df_topix15min_volume=pd.DataFrame(topix15min_volume,index=train15min_data_date)
df_MinMax15min_index=pd.DataFrame(MinMax15min_index,index=train15min_data_date)
df_MinMax15min_close=pd.DataFrame(MinMax15min_close,index=train15min_data_date)
#可視化
#plt.plot(df_topix15min_index.T)
#plt.plot(df_MinMax15min_index.T)

In [26]:
topix_train=topix_index#+topix15min_index
df_topix_train=pd.DataFrame(topix_train,index=train_dates[:48])
#len(df_topix_train)
all_file=file_names[:48]#+file_15min
len(all_file)

48

In [28]:
#DTW計算
from fastdtw import fastdtw
distance=[]
path=[]
for i in range(len(topix_train)):
    print(len(topix_train)-i)
    for j in range(len(topix_train)):
        dist,pa=fastdtw(topix_train[i], topix_train[j],dist=euclidean)
        distance.append(dist)
        path.append(pa)
#listの分割　
def split_list(l, n):
    """
    リストをサブリストに分割する
    :param l: リスト
    :param n: サブリストの要素数
    :return: 
    """
    for i in range(0, len(l), n):
        yield l[i:i + n]
 
result = list(split_list(distance, len(topix_train)))
#DTW行列
c=np.array(result)
#df_c=pd.DataFrame(c)
#df_c.to_csv("train_data.csv",encoding="shift_jis")

#k-medoids法
from sklearn_extra.cluster import KMedoids
import numpy as np
kmedoid=KMedoids(n_clusters=30, metric='euclidean', method='alternate', init='heuristic', max_iter=300, random_state=None).fit(c)
#cluster追加
df_topix_train["cluster"]=kmedoid.labels_

48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1


In [29]:
df_topix_train.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,cluster
2012-01-01,1.0,0.990969,0.981978,0.985114,0.987187,0.978681,0.988708,0.984576,0.989219,0.996891,...,1.029099,1.024415,1.018870,1.016528,NaN,NaN,NaN,NaN,NaN,1
2012-02-01,1.0,1.005924,1.003602,1.015687,1.019539,1.032165,1.035002,1.027851,1.031294,1.038049,...,1.088976,1.094187,1.100704,1.101971,1.106232,1.102908,NaN,NaN,NaN,29
2012-03-01,1.0,1.007552,1.001587,0.994961,0.989381,1.005556,1.020648,1.016524,1.016584,1.030750,...,1.036715,1.025242,1.024389,1.049162,1.039553,1.031508,1.027431,NaN,NaN,2
2012-04-01,1.0,0.994124,0.975831,0.972572,0.950517,0.950213,0.941347,0.946066,0.952608,0.938999,...,0.945669,0.939127,0.945611,0.946323,0.939513,NaN,NaN,NaN,NaN,27
2012-05-01,1.0,1.004281,0.977922,0.983635,0.970031,0.969512,0.960595,0.958442,0.946687,0.935895,...,0.913970,0.914831,0.914654,0.920886,0.916566,0.911335,NaN,NaN,NaN,27


In [30]:
#翌月の月末価格の上昇下落ラベル

#月末終値追加
m=[]
for file in all_file:
    data=pd.read_csv(file,header=0)
    m.append(int(data["Close"].tail(1).item()))
df_topix_train["month_close"]=m
#目的変数Upを追加する(翌日の月末が上がる場合1、それ以外は0とする)、'delta_Close'カラムの削除
df_topix_train['Up']=df_topix_train['cluster']
for i in range(len(df_topix_train['month_close'])-1):
    if df_topix_train['month_close'][i] < df_topix_train['month_close'][i+1]:
        df_topix_train['Up'][i]=1
    else:
        df_topix_train['Up'][i]=0
#df = df.drop('delta_Close', axis=1)
#df_test=df_topix_index[["cluster","month_close","Up"]]

In [31]:
df_topix_train.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,cluster,month_close,Up
2012-01-01,1.0,0.990969,0.981978,0.985114,0.987187,0.978681,0.988708,0.984576,0.989219,0.996891,...,1.018870,1.016528,NaN,NaN,NaN,NaN,NaN,1,755,1
2012-02-01,1.0,1.005924,1.003602,1.015687,1.019539,1.032165,1.035002,1.027851,1.031294,1.038049,...,1.100704,1.101971,1.106232,1.102908,NaN,NaN,NaN,29,835,1
2012-03-01,1.0,1.007552,1.001587,0.994961,0.989381,1.005556,1.020648,1.016524,1.016584,1.030750,...,1.024389,1.049162,1.039553,1.031508,1.027431,NaN,NaN,2,854,0
2012-04-01,1.0,0.994124,0.975831,0.972572,0.950517,0.950213,0.941347,0.946066,0.952608,0.938999,...,0.945611,0.946323,0.939513,NaN,NaN,NaN,NaN,27,804,0
2012-05-01,1.0,1.004281,0.977922,0.983635,0.970031,0.969512,0.960595,0.958442,0.946687,0.935895,...,0.914654,0.920886,0.916566,0.911335,NaN,NaN,NaN,27,719,1


In [214]:
df_topix_train.to_csv("topix_train30.csv",encoding="shift_jis")

<h1>検証フェーズ</h1>

In [32]:
test_data_date=["2016-01-01","2016-02-01","2016-03-01","2016-04-01","2016-05-01","2016-06-01","2016-07-01","2016-08-01","2016-09-01","2016-10-01","2016-11-01","2016-12-01",
             "2017-01-01","2017-02-01","2017-03-01","2017-04-01","2017-05-01","2017-06-01","2017-07-01","2017-08-01","2017-09-01","2017-10-01","2017-11-01","2017-12-01",
             "2018-01-01","2018-02-01","2018-03-01","2018-04-01","2018-05-01","2018-06-01","2018-07-01","2018-08-01","2018-09-01","2018-10-01","2018-11-01","2018-12-01",
             "2019-01-01","2019-02-01","2019-03-01","2019-04-01","2019-05-01","2019-06-01","2019-07-01","2019-08-01","2019-09-01","2019-10-01","2019-11-01","2019-12-01",
             "2020-01-01","2020-02-01","2020-03-01","2020-04-01","2020-05-01","2020-06-01","2020-07-01","2020-08-01","2020-09-01","2020-10-01","2020-11-01","2020-12-01"]
test_file_names=list(map(lambda s: s + '.csv',test_data_date ))
test_topix_close=[]
test_topix_index=[]
test_topix_volume=[]
test_MinMax_close=[]
test_MinMax_index=[]
for file in test_file_names:
    data=pd.read_csv(file,header=0)
    #正規化のクラスを準備
    ms = MinMaxScaler()
    #特徴量の最大値と最小値を計算し変換
    X = np.array(data["Close"]).reshape(-1,1)
    data["MinMax_Close"] = ms.fit_transform(X)#最小値0最大値1の正規化
    a=data["Close"][0]#日の始値
    b=data["MinMax_Close"][0]#日の始値
    data["MinMax_index"]=data["Close"]
    data["index"]=data["Close"]
    for i in range(len(data.index)):
        data["index"][i]=data["Close"][i]/a
        data["MinMax_index"][i]=data["MinMax_Close"][i]/b
    test_topix_close.append(data["Close"].tail(1).values)
    test_topix_index.append(data["index"].values)
    test_topix_volume.append(data["Volume"].values)
    test_MinMax_close.append(data["MinMax_Close"].values)
    test_MinMax_index.append(data["MinMax_index"].values)
df_test_topix_index=pd.DataFrame(test_topix_index,index=test_data_date)
df_test_topix_close=pd.DataFrame(test_topix_close,index=test_data_date)
df_test_topix_volume=pd.DataFrame(test_topix_volume,index=test_data_date)
df_test_MinMax_close=pd.DataFrame(test_MinMax_close,index=test_data_date)
df_test_MinMax_index=pd.DataFrame(test_MinMax_index,index=test_data_date)
#可視化　
#plt.plot(df_test_topix_index.T)
#plt.plot(df_MinMax_index.T)

In [158]:
#test_topix_index.append(test[0])
for i in range(10):
    test_topix.pop(0)
len(test_topix)

1038

In [33]:
test_topix=topix_train
df_test_topix=pd.DataFrame(test_topix,index=train_dates[:48])
#test_topix
#len(test_topix_index)
len(test_topix)

48

In [34]:
test_distance=[]
test_path=[]
#DTW
for i in range(len(test_topix_index)):#テストデータ数分繰り返す 2016-01-01から2020-12-01までの60ヵ月分
    #test_topix.append(test_topix_index[i])#学習データに追加
    print(len(test_topix_index)-i)
    for j in range(len(test_topix)):#データ数分繰り返す1039.40...
        test_dist,test_pa=fastdtw(test_topix_index[i], test_topix[j],dist=euclidean)
        test_distance.append(test_dist)
        test_path.append(test_pa)
test_result = list(split_list(test_distance,len(test_topix)))

#DTW行列
test_c=np.array(test_result)
#df_c=pd.DataFrame(c,index=datas,columns=datas)
#df_c.to_csv("dtw.csv",encoding="shift_jis")
#k-medoids法 
test_kmedoid=KMedoids(n_clusters=30, 
                       metric='euclidean', 
                       method='alternate', 
                       init='heuristic', 
                       max_iter=300, 
                       random_state=None).fit(test_c)
#cluster追加
#test_df_topix=pd.DataFrame(test_topix)
df_test_topix_index["cluster"]=test_kmedoid.labels_
df_test_topix_index.groupby('cluster').count().iloc[:,1]

60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1


cluster
0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    2
19    1
20    1
21    7
22    1
23    2
24    2
25    1
26    8
27    3
28    6
29    8
Name: 1, dtype: int64

In [40]:
df_topix_train.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,cluster,month_close,Up
2012-01-01,1.0,0.990969,0.981978,0.985114,0.987187,0.978681,0.988708,0.984576,0.989219,0.996891,...,1.018870,1.016528,NaN,NaN,NaN,NaN,NaN,1,755,1
2012-02-01,1.0,1.005924,1.003602,1.015687,1.019539,1.032165,1.035002,1.027851,1.031294,1.038049,...,1.100704,1.101971,1.106232,1.102908,NaN,NaN,NaN,29,835,1
2012-03-01,1.0,1.007552,1.001587,0.994961,0.989381,1.005556,1.020648,1.016524,1.016584,1.030750,...,1.024389,1.049162,1.039553,1.031508,1.027431,NaN,NaN,2,854,0
2012-04-01,1.0,0.994124,0.975831,0.972572,0.950517,0.950213,0.941347,0.946066,0.952608,0.938999,...,0.945611,0.946323,0.939513,NaN,NaN,NaN,NaN,27,804,0
2012-05-01,1.0,1.004281,0.977922,0.983635,0.970031,0.969512,0.960595,0.958442,0.946687,0.935895,...,0.914654,0.920886,0.916566,0.911335,NaN,NaN,NaN,27,719,1


In [41]:
df=pd.concat([df_topix_train,df_test_topix_index])

In [98]:
df_topix_train.groupby('cluster').count().iloc[:,1]

cluster
0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    6
20    3
21    2
22    2
23    1
24    1
25    1
26    1
27    7
28    3
29    2
Name: 1, dtype: int64

In [81]:
#翌月の月末価格の上昇下落ラベル
#月末終値追加
m=[]
return_graph=[]
sum_return=0
for file in test_file_names:
    data=pd.read_csv(file,header=0)
    m.append(int(data["Close"].tail(1).item()))
df_test_topix_index["month_close"]=df_test_topix_index['cluster']
df_test_topix_index["month_close"]=m

#正解ラベルの付与
#目的変数Upを追加する(翌日の月末が上がる場合1、それ以外は0とする)、
df_test_topix_index['Up']=str(df_test_topix_index['cluster'])

In [93]:
ruiseki_return=0
for i in range(len(df_test_topix_index['month_close'])-1):
    if df_test_topix_index['month_close'][i] < df_test_topix_index['month_close'][i+1]:
        df_test_topix_index['Up'][i]="up"
    else:
        df_test_topix_index['Up'][i]="down"
    #df = df.drop('delta_Close', axis=1)
    #df_test=test_df_topix[["cluster","month_close","Up"]]
    
    for j in range(30):#クラスタ数回
        if df_test_topix_index["cluster"][i]==j:#クラスタ数の特定
        #クラスタ内の上昇（下落）ラベルの多数決
            n=df[df["cluster"]==j]
            n.reset_index(drop=True,inplace=True)
            n["Time"] = pd.to_datetime(df_test_topix_index["index"])
            up=int((n['Up']=="up").sum())#上昇ラベルの総数
            down=int((n["Up"]=="down").sum())#下落ラベルの総数
            if up>down:#上昇ラベルの方が多い場合
                #df_test_topix_index["pred_Up"]=str(df_test_topix_index["cluster"])
                df_test_topix_index["pred_Up"][i]="up"
                position=df_test_topix_index["month_close"][i]#月末価格で購入
                #next_month=df_test_topix_index["Time"][i]+relativedelta(months=+1)#翌月
                #m_date=(n["Time"][j]+relativedelta(months=+1)).strftime('%Y-%m-%d')
                #m_date=pd.to_datetime(m_date)
                #df_test_topix_index.reset_index(inplace=True)
                #df_test_topix_index["Time"] = pd.to_datetime(df_test_topix_index.index)
                #n.set_index("index", inplace = True)
                #df_test_topix_index["Time"][0]+relativedelta(months=+1)
                #df_train["month_close"][df_train["Time"] [0] +relativedelta(months=+1).fromtimestamp(ts)]
                #if next_month:#翌月の場合
                rikaku=df_test_topix_index["month_close"][i+1]#翌月価格
                profit=rikaku-position#翌月価格で売った時の収益
                month_return=(rikaku/position)-1#リターン％
                return_graph.append(month_return)
                ruiseki_return=(ruiseki_return+month_return+1)*100#累積収益率
                #sum=sum+profit#利益の合計
                #print("利益=",sum)
            #s.append(sum)#毎月の利益
            elif up<down:
                #df_test_topix_index["pred_Up"]=str(df_test_topix_index["cluster"])
                df_test_topix_index["pred_Up"][i]="down"
                position=n["month_close"][i]#月末価格で売却
                    #df_test_topix_index["Time"] = pd.to_datetime(df_test_topix_index.index)
                #next_month=n["Time"][i]+relativedelta(months=+1)#翌月
                #if next_month:#翌月の場合
                rikaku=df_test_topix_index["month_close"][s]#翌月価格
                profit=position-rikaku#翌月価格で買った時の収益
                month_return=(rikaku/position)-1#リターン％
                return_graph.append(month_return)
                ruiseki_return=(ruiseki_return+month_return+1)*100#累積収益率
                #ruiseki_return=(1+month_return).cumprod()#累積収益率
                #sum=sum+profit#利益の合計
                #print("利益=",sum)
            #s.append(sum)  

In [95]:
n["Up"]

0     0
1     1
2     1
3    28
4    28
5    28
6    28
7    28
8    28
Name: Up, dtype: int64

In [78]:
#翌月の月末価格の上昇下落ラベル
#月末終値追加
m=[]
return_graph=[]
sum_return=0
for file in test_file_names:
    data=pd.read_csv(file,header=0)
    m.append(int(data["Close"].tail(1).item()))
df_test_topix_index["month_close"]=df_test_topix_index['cluster']
df_test_topix_index["month_close"]=m

#正解ラベルの付与
#目的変数Upを追加する(翌日の月末が上がる場合1、それ以外は0とする)、
#df_test_topix_index['Up']=df_test_topix_index['cluster']

for i in range(len(df_test_topix_index['month_close'])-1):
    if df_test_topix_index['month_close'][i] < df_test_topix_index['month_close'][i+1]:
        df_test_topix_index['Up'][i]="up"
    else:
        df_test_topix_index['Up'][i]="down"
    #df = df.drop('delta_Close', axis=1)
    #df_test=test_df_topix[["cluster","month_close","Up"]]
    
    for j in range(30):#クラスタ数回
        if df_test_topix_index["cluster"][i]==j:#クラスタ数の特定
        #クラスタ内の上昇（下落）ラベルの多数決
            n=df[df["cluster"]==j]
            df_test_topix_index.reset_index(drop=True,inplace=True)
            df_test_topix_index["Time"] = pd.to_datetime(df_test_topix_index["index"])
            up=int((n['Up']=="up").sum())#上昇ラベルの総数
            down=int((n["Up"]=="down").sum())#下落ラベルの総数
            print(n['Up'])
            
            #print("up",up)
            #print("down",down)
            if up>down:#上昇ラベルの方が多い場合
                #df_test_topix_index["pred_Up"]=df_test_topix_index["cluster"]
                df_test_topix_index["pred_Up"][i]="up"
                position=df_test_topix_index["month_close"][i]#月末価格で購入
                #next_month=df_test_topix_index["Time"][i]+relativedelta(months=+1)#翌月
                #m_date=(n["Time"][j]+relativedelta(months=+1)).strftime('%Y-%m-%d')
                #m_date=pd.to_datetime(m_date)
                #df_test_topix_index.reset_index(inplace=True)
                #df_test_topix_index["Time"] = pd.to_datetime(df_test_topix_index.index)
                #n.set_index("index", inplace = True)
                #df_test_topix_index["Time"][0]+relativedelta(months=+1)
                #df_train["month_close"][df_train["Time"] [0] +relativedelta(months=+1).fromtimestamp(ts)]
                #if next_month:#翌月の場合
                rikaku=df_test_topix_index["month_close"][i+1]#翌月価格
                profit=rikaku-position#翌月価格で売った時の収益
                month_return=(rikaku/position)-1#リターン％
                return_graph.append(month_return)
                ruiseki_return=(ruiseki_return+month_return+1)*100#累積収益率
                #sum=sum+profit#利益の合計
                #print("利益=",sum)
            #s.append(sum)#毎月の利益
            elif up<down:
                #df_test_topix_index["pred_Up"]=df_test_topix_index["cluster"]
                df_test_topix_index["pred_Up"][i]="down"
                position=n["month_close"][i]#月末価格で売却
                    #df_test_topix_index["Time"] = pd.to_datetime(df_test_topix_index.index)
                #next_month=n["Time"][i]+relativedelta(months=+1)#翌月
                #if next_month:#翌月の場合
                rikaku=df_test_topix_index["month_close"][s]#翌月価格
                profit=position-rikaku#翌月価格で買った時の収益
                month_return=(rikaku/position)-1#リターン％
                return_graph.append(month_return)
                ruiseki_return=(ruiseki_return+month_return+1)*100#累積収益率
                #ruiseki_return=(1+month_return).cumprod()#累積収益率
                #sum=sum+profit#利益の合計
                #print("利益=",sum)
            #s.append(sum)  

2012-02-01     1
2013-04-01     0
2016-01-01     0
2016-02-01    29
2016-06-01    29
2018-02-01    29
2018-10-01    29
2018-12-01    29
2019-08-01    29
2020-03-01    29
Name: Up, dtype: int64
2012-02-01     1
2013-04-01     0
2016-01-01     0
2016-02-01    29
2016-06-01    29
2018-02-01    29
2018-10-01    29
2018-12-01    29
2019-08-01    29
2020-03-01    29
Name: Up, dtype: int64
2012-06-01     0
2012-12-01     1
2015-02-01     1
2016-03-01    28
2019-01-01    28
2019-09-01    28
2020-04-01    28
2020-08-01    28
2020-11-01    28
Name: Up, dtype: int64
2012-04-01     0
2012-05-01     1
2012-07-01     0
2014-04-01     1
2014-10-01     1
2015-08-01     0
2015-12-01    27
2016-04-01    27
2016-07-01    27
2020-05-01    27
Name: Up, dtype: int64
2013-05-01     0
2013-07-01     0
2016-05-01    21
2016-10-01    21
2016-12-01    21
2017-10-01    21
2018-04-01    21
2018-07-01    21
2019-06-01    21
Name: Up, dtype: int64
2012-02-01     1
2013-04-01     0
2016-01-01     0
2016-02-01    29
2

In [61]:
acc=0
bunbo=0
for i in range(60):
    if df_test_topix_index["pred_Up"][i]=="up" or df_test_topix_index["pred_Up"][i]=="down":
        bunbo=bunbo+1
        if df_test_topix_index["pred_Up"][i]==df_test_topix_index["Up"][i]:
            acc=acc+1
print("累積収益率：",ruiseki_return,"%")
print("正解率：",acc/bunbo)

累積収益率： 9.162122805614159e+117 %
正解率： 0.6271186440677966


In [76]:
df_test_topix_index

,level_0,index,0,1,2,3,4,5,6,7,...,18,19,20,21,22,cluster,month_close,Up,Time,pred_Up
0,0,2016-01-01,1.0,0.996714,0.986202,0.965734,0.958700,0.928647,0.955235,0.931694,...,NaN,NaN,NaN,NaN,NaN,29,1432,down,2016-01-01,down
1,1,2016-02-01,1.0,0.992732,0.961440,0.949503,0.935939,0.943760,0.891746,0.864829,...,0.887316,NaN,NaN,NaN,NaN,29,1297,up,2016-02-01,up
2,2,2016-03-01,1.0,1.037499,1.052444,1.057287,1.046947,1.036046,1.024215,1.039467,...,1.059016,1.042634,1.035646,NaN,NaN,28,1347,down,2016-03-01,up
3,3,2016-04-01,1.0,1.001007,0.974620,0.974143,0.977901,0.989465,0.983395,0.998425,...,1.063701,1.030083,NaN,NaN,NaN,27,1340,up,2016-04-01,up
4,4,2016-05-01,1.0,0.998738,1.005154,1.026878,1.026416,1.028701,1.015562,1.016685,...,NaN,NaN,NaN,NaN,NaN,21,1379,down,2016-05-01,up
5,5,2016-06-01,1.0,0.977784,0.981763,0.978239,0.984362,0.991851,0.981895,0.976984,...,0.899924,0.899087,0.916025,0.914652,NaN,29,1245,up,2016-06-01,up
6,6,2016-07-01,1.0,1.001754,0.983865,0.977400,0.964478,1.001076,1.024943,1.036526,...,1.054447,NaN,NaN,NaN,NaN,27,1322,up,2016-07-01,up
7,7,2016-08-01,1.0,0.983636,0.962287,0.970617,0.968279,0.987669,0.996717,0.994704,...,0.974331,0.993501,0.993176,1.005833,NaN,26,1329,down,2016-08-01,up
8,8,2016-09-01,1.0,1.002527,1.011365,1.009085,1.006408,1.004845,0.989322,0.989240,...,0.989083,NaN,NaN,NaN,NaN,18,1322,up,2016-09-01,up
9,9,2016-10-01,1.0,1.007131,1.012843,1.017442,1.014947,1.019260,1.008740,1.008710,...,1.046358,1.046817,NaN,NaN,NaN,21,1393,up,2016-10-01,up


In [ ]:
#変動率
#テクニカル指標
#月、週、日単位の平均、標準偏差
#移動平均線のパターン分析
#トレンド・季節性・周期性・不規則性→不規則性を削除もしくは不規則性のトレンドのみに適用

In [ ]:
# 終値の前日比の追加
df_shift = df.shift(1)

df['Close_ratio'] = (df['Close'] - df_shift['Close']) / df_shift['Close']
df

# 始値と終値の差分を追加
df['Body'] = df['Open'] - df['Close']
df

,weeks,weekday,High,Low,Open,Close,Up,Close_ratio
Date,,,,,,,,
1965-01-05,0,1,1.163391e+05,1.163391e+05,1.163391e+05,1.163391e+05,1,NaN
1965-01-06,0,2,1.169191e+05,1.169191e+05,1.169191e+05,1.169191e+05,1,0.004985
1965-01-07,0,3,1.178700e+05,1.178700e+05,1.178700e+05,1.178700e+05,1,0.008133
1965-01-08,0,4,1.189948e+05,1.189948e+05,1.189948e+05,1.189948e+05,1,0.009543
1965-01-12,1,1,1.191900e+05,1.191900e+05,1.191900e+05,1.191900e+05,0,0.001640
...,...,...,...,...,...,...,...,...
2021-10-15,2962,4,2.690117e+06,2.657175e+06,2.662822e+06,2.688848e+06,0,0.018133
2021-10-18,2963,0,2.695851e+06,2.675507e+06,2.691178e+06,2.684855e+06,1,-0.001485
2021-10-19,2963,1,2.707705e+06,2.689472e+06,2.693348e+06,2.702436e+06,1,0.006548


## 検証データによる予測と精度検証

In [ ]:
#参考文献
k-medoids
#https://python-kmedoids.readthedocs.io/en/latest/